In [3]:
%pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 54.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.7/68.7 MB 36.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 72.6 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully uninstalled flatbuffers-1.12
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
  Attempting uninstall: attrs
    Found existing installation: attrs 18.2.0
    Uninstalling attrs-18.2.0:
      Successfully uninstalled attrs-18.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts

In [4]:
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import load_model
from midiutil import MIDIFile
from pydub import AudioSegment
from pydub.playback import play

# Load the pre-trained emotion recognition model
model = load_model('/notebooks/AI-forMedia-project-23-24/model.h5')

# Initialize Mediapipe Face Mesh
mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection()

# Initialize MIDIFile for music generation
midi = MIDIFile(1)
track = 0
time = 0
midi.addTrackName(track, time, "Emotion Track")
midi.addTempo(track, time, 120)

# Labels corresponding to your model's output (adjust based on your model's training labels)
emotion_labels = ['happy', 'sad', 'angry', 'neutral']  # Update based on your model

# Start video capture
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    # Convert the image to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = face_detection.process(rgb_frame)

    if result.detections:
        for detection in result.detections:
            # Get the bounding box coordinates of the detected face
            bboxC = detection.location_data.relative_bounding_box
            ih, iw, _ = frame.shape
            x, y, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), int(bboxC.width * iw), int(bboxC.height * ih)
            face = frame[y:y+h, x:x+w]

            # Preprocess the face image for emotion recognition model
            face_resized = cv2.resize(face, (48, 48))  # Resize to match the input size of your model
            face_gray = cv2.cvtColor(face_resized, cv2.COLOR_BGR2GRAY)  # Convert to grayscale if required by your model
            face_array = np.expand_dims(face_gray, axis=0)  # Add batch dimension
            face_array = face_array / 255.0  # Normalize pixel values

            # Predict emotion using the model
            predictions = model.predict(face_array)
            emotion_idx = np.argmax(predictions)
            emotion = emotion_labels[emotion_idx]

            # Map emotion to a music note
            if emotion == "happy":
                note = 60  # C4
            elif emotion == "sad":
                note = 48  # C3
            elif emotion == "angry":
                note = 67  # G4
            else:
                note = 60  # Default C4 (for neutral or other emotions)

            # Generate music based on emotion
            duration = 1  # 1 beat duration
            volume = 100  # volume level
            midi.addNote(track, 0, note, time, duration, volume)

    cv2.imshow("Facial Expression Analysis", frame)
    if cv2.waitKey(5) & 0xFF == 27:  # Press 'ESC' to exit
        break

cap.release()
cv2.destroyAllWindows()

# Save the generated MIDI file
with open("emotion_music.mid", "wb") as output_file:
    midi.writeFile(output_file)

# Play the generated MIDI file using Pydub (convert MIDI to audio)
midi_audio = AudioSegment.from_file("emotion_music.mid", format="mid")
play(midi_audio)


TypeError: Descriptors cannot be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates